In [2]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!unzip '/content/gdrive/MyDrive/Rotation.zip' -d '/content/gdrive/MyDrive/'

Streaming output truncated to the last 5000 lines.
  inflating: /content/gdrive/MyDrive/Rotation/781647resized2rotated.jpg  
  inflating: /content/gdrive/MyDrive/Rotation/967884resized1rotated.jpg  
  inflating: /content/gdrive/MyDrive/Rotation/459013resized2rotated.jpg  
  inflating: /content/gdrive/MyDrive/Rotation/905904resized3rotated.jpg  
  inflating: /content/gdrive/MyDrive/Rotation/770027resized2rotated.jpg  
  inflating: /content/gdrive/MyDrive/Rotation/395579resized4rotated.jpg  
  inflating: /content/gdrive/MyDrive/Rotation/627046resized4rotated.jpg  
  inflating: /content/gdrive/MyDrive/Rotation/887648resized3rotated.jpg  
  inflating: /content/gdrive/MyDrive/Rotation/226387resized2rotated.jpg  
  inflating: /content/gdrive/MyDrive/Rotation/656905resized3rotated.jpg  
  inflating: /content/gdrive/MyDrive/Rotation/175842resized1rotated.jpg  
  inflating: /content/gdrive/MyDrive/Rotation/288091resized1rotated.jpg  
  inflating: /content/gdrive/MyDrive/Rotation/520800resized3r

In [4]:
import keras
from keras.models import Sequential, Model
from keras.layers.core import Flatten, Dense, Dropout, Lambda, Reshape
from keras.layers import Input
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Conv2D, MaxPooling2D, Activation
from keras.optimizers import RMSprop

In [5]:
def ConvLayers(layers, model, filters):
  for i in range(layers):
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(filters, 3,3,activation='relu'))
  model.add(MaxPooling2D((2,2),strides=(1,1),padding="same"))

In [6]:
def FCLayers(model):
  model.add(Dense(4096, activation='relu'))
  model.add(Dropout(0.5))

In [7]:
def VGG16_arc():
  model=Sequential()
  ConvLayers(2,model,64)
  ConvLayers(2,model,128)
  ConvLayers(3,model,256)
  ConvLayers(3,model,512)
  ConvLayers(3,model,512)
  model.add(Flatten())
  FCLayers(model)
  FCLayers(model)
  model.add(Dense(37,activation='sigmoid'))
  #model.add(Dense(3,activation='softmax'))
  return model

In [8]:
#Model compilation
optimizer = RMSprop(lr=1e-6) #can be later tweaked
model = VGG16_arc()
model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy']) 
#model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [9]:
model.build(input_shape=(None,224,224,3))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d (ZeroPadding2 (None, 226, 226, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 75, 75, 64)        1792      
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 77, 77, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 25, 25, 64)        36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 25, 25, 64)        0         
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 27, 27, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 128)         7

In [10]:
import pandas as pd
df=pd.read_csv('/content/sample_data/Galaxy/trainingdffinal.csv')
df.head(10)

,Unnamed: 0,GalaxyID,Class1.1,Class1.2,Class1.3,Class2.1,Class2.2,Class3.1,Class3.2,Class4.1,Class4.2,Class5.1,Class5.2,Class5.3,Class5.4,Class6.1,Class6.2,Class7.1,Class7.2,Class7.3,Class8.1,Class8.2,Class8.3,Class8.4,Class8.5,Class8.6,Class8.7,Class9.1,Class9.2,Class9.3,Class10.1,Class10.2,Class10.3,Class11.1,Class11.2,Class11.3,Class11.4,Class11.5,Class11.6
0,0,100008resized1rotated.jpg,0.383147,0.616853,0.000000,0.000000,0.616853,0.038452,0.578401,0.418398,0.198455,0.000000,0.104752,0.512101,0.000000,0.054453,0.945547,0.201463,0.181684,0.000000,0.0,0.027227,0.000000,0.027227,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.279952,0.138445,0.000000,0.000000,0.092886,0.000000,0.000000,0.000000,0.325512
1,1,100023resized1rotated.jpg,0.327001,0.663777,0.009222,0.031178,0.632599,0.467370,0.165229,0.591328,0.041271,0.000000,0.236781,0.160941,0.234877,0.189149,0.810851,0.000000,0.135082,0.191919,0.0,0.000000,0.140353,0.000000,0.048796,0.000000,0.000000,0.012414,0.0,0.018764,0.000000,0.131378,0.459950,0.000000,0.591328,0.000000,0.000000,0.000000,0.000000
2,2,100053resized1rotated.jpg,0.765717,0.177352,0.056931,0.000000,0.177352,0.000000,0.177352,0.000000,0.177352,0.000000,0.117790,0.059562,0.000000,0.000000,1.000000,0.000000,0.741864,0.023853,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,3,100078resized1rotated.jpg,0.693377,0.238564,0.068059,0.000000,0.238564,0.109493,0.129071,0.189098,0.049466,0.000000,0.000000,0.113284,0.125280,0.320398,0.679602,0.408599,0.284778,0.000000,0.0,0.000000,0.096119,0.096119,0.000000,0.128159,0.000000,0.000000,0.0,0.000000,0.094549,0.000000,0.094549,0.189098,0.000000,0.000000,0.000000,0.000000,0.000000
4,4,100090resized1rotated.jpg,0.933839,0.000000,0.066161,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.029383,0.970617,0.494587,0.439252,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.029383,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,5,100122resized1rotated.jpg,0.738832,0.238159,0.023009,0.000000,0.238159,0.000000,0.238159,0.000000,0.238159,0.000000,0.000000,0.238159,0.000000,0.197930,0.802070,0.066807,0.663691,0.008335,0.0,0.000000,0.049482,0.098965,0.049482,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,6,100123resized1rotated.jpg,0.462492,0.456033,0.081475,0.000000,0.456033,0.000000,0.456033,0.000000,0.456033,0.000000,0.000000,0.456033,0.000000,0.687647,0.312353,0.388158,0.074334,0.000000,0.0,0.000000,0.000000,0.000000,0.213858,0.473789,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,7,100128resized1rotated.jpg,0.687783,0.288344,0.023873,0.000000,0.288344,0.069098,0.219246,0.000000,0.288344,0.067228,0.123625,0.027836,0.069656,0.473888,0.526112,0.482483,0.205300,0.000000,0.0,0.067766,0.033646,0.000000,0.304710,0.067766,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,8,100134resized1rotated.jpg,0.021834,0.976952,0.001214,0.021751,0.955201,0.313077,0.642124,0.546491,0.408711,0.160096,0.760688,0.034417,0.000000,0.611499,0.388501,0.010917,0.010917,0.000000,0.0,0.032377,0.064143,0.450225,0.000000,0.032377,0.032377,0.000000,0.0,0.021751,0.207253,0.152044,0.187194,0.000000,0.000000,0.054649,0.081974,0.081974,0.327894
9,9,100143resized1rotated.jpg,0.269843,0.730157,0.000000,0.730157,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.410635,0.589365,0.000000,0.000000,0.269843,0.0,0.000000,0.034117,0.000000,0.136878,0.239640,0.000000,0.561429,0.0,0.168728,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [17]:
classes = ['Class1.1', 'Class1.2', 'Class1.3', 'Class2.1', 'Class2.2', 'Class3.1','Class3.2', 'Class4.1', 'Class4.2', 'Class5.1', 'Class5.2', 'Class5.3','Class5.4', 'Class6.1', 'Class6.2', 'Class7.1', 'Class7.2', 'Class7.3','Class8.1', 'Class8.2', 'Class8.3', 'Class8.4', 'Class8.5', 'Class8.6', 'Class8.7', 'Class9.1', 'Class9.2', 'Class9.3', 'Class10.1', 'Class10.2', 'Class10.3', 'Class11.1', 'Class11.2', 'Class11.3', 'Class11.4', 'Class11.5', 'Class11.6'
]

In [23]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagenerator = ImageDataGenerator(validation_split=0.2)

train_generator = datagenerator.flow_from_dataframe(
    dataframe=df,
    directory="/content/gdrive/MyDrive/Rotation/",
    x_col="GalaxyID",
    y_col=classes,
    subset="training",
    batch_size=64,
    seed=123,
    shuffle=True,
    class_mode="raw",
    target_size=(224, 224))
valid_generator = datagenerator.flow_from_dataframe(
    dataframe=df,
    directory="/content/gdrive/MyDrive/Rotation/",
    x_col="GalaxyID",
    y_col=classes,
    subset="validation",
    batch_size=64,
    seed=123,
    shuffle=True,
    class_mode="raw",
    target_size=(224, 224))
train_step_size = train_generator.n // train_generator.batch_size
validation_step_size = valid_generator.n // valid_generator.batch_size

Found 197050 validated image filenames.
Found 49262 validated image filenames.


In [24]:
train_step_size

3078

In [25]:
validation_step_size

769

In [26]:
import h5py

In [27]:
import timeit
import matplotlib.pyplot as plt

In [ ]:
from keras import callbacks
start = timeit.default_timer()
earlystopping = callbacks.EarlyStopping(monitor ="val_loss", 
                                        mode ="auto", patience = 5, verbose=2, 
                                        restore_best_weights = True)

checkpoint = callbacks.ModelCheckpoint(
    filepath='/content/gdrive/MyDrive/Rotation/weights.hdf5', verbose=2, save_best_only=True)

classifier = model.fit(train_generator,
                       steps_per_epoch=train_step_size,
                       epochs=50,
                       validation_data=valid_generator,
                       validation_steps=validation_step_size,
                       callbacks =[checkpoint,earlystopping],
                       verbose=2,
                       )
end = timeit.default_timer()
print("Time Taken to run the model:",end - start, "seconds")
plot_history(classifier)

Epoch 1/50
3078/3078 - 690s - loss: 0.1039 - accuracy: 0.5933 - val_loss: 0.0274 - val_accuracy: 0.5961

Epoch 00001: val_loss improved from inf to 0.02739, saving model to /content/gdrive/MyDrive/Rotation/weights.hdf5
Epoch 2/50
3078/3078 - 660s - loss: 0.0277 - accuracy: 0.5958 - val_loss: 0.0266 - val_accuracy: 0.5960

Epoch 00002: val_loss improved from 0.02739 to 0.02659, saving model to /content/gdrive/MyDrive/Rotation/weights.hdf5
Epoch 3/50
3078/3078 - 596s - loss: 0.0270 - accuracy: 0.5961 - val_loss: 0.0264 - val_accuracy: 0.5959

Epoch 00003: val_loss improved from 0.02659 to 0.02636, saving model to /content/gdrive/MyDrive/Rotation/weights.hdf5
Epoch 4/50
3078/3078 - 592s - loss: 0.0268 - accuracy: 0.5961 - val_loss: 0.0262 - val_accuracy: 0.5959

Epoch 00004: val_loss improved from 0.02636 to 0.02621, saving model to /content/gdrive/MyDrive/Rotation/weights.hdf5
Epoch 5/50
3078/3078 - 583s - loss: 0.0265 - accuracy: 0.5961 - val_loss: 0.0260 - val_accuracy: 0.5960

Epoch 0